<a href="https://colab.research.google.com/github/RamSaw/deep-reinforcement-learning/blob/master/RL_HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym[box2d]

     |████████████████████████████████| 450kB 2.8MB/s 


In [0]:
import os
import random

import numpy as np
import torch
from torch import nn

SEED = 11  # 627, 8, 11


def make_reproducible(seed, make_cuda_reproducible):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if make_cuda_reproducible:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


make_reproducible(SEED, make_cuda_reproducible=False)


def transform_state(state):
    return torch.tensor(state)


class Agent:
    def __init__(self):
        self.model = Agent.generate_model()
        self.model.load_state_dict(torch.load(__file__[:-8] + "/agent.pkl"))
        self.model.to(torch.device("cpu"))
        self.model.eval()

    @staticmethod
    def generate_model():
        linears = [nn.Linear(8, 128), nn.Linear(128, 64), nn.Linear(64, 4)]
        #for l in linears:
        #    nn.init.uniform_(l.weight)
        #    nn.init.uniform_(l.bias)
        return nn.Sequential(
                linears[0],
                nn.ReLU(),
                linears[1],
                nn.ReLU(),
                linears[2])

    def act(self, state):
        with torch.no_grad():
            state = transform_state(state)
            return torch.argmax(self.model(state)).item()

    def reset(self):
        pass

In [0]:
import math
import random
import time
from collections import deque, namedtuple

import numpy as np
import torch
import torch.nn.functional as F
from gym import make
from torch import optim


N_STEP = 1
GAMMA = 0.99
LR = 5e-4
BATCH_SIZE = 64
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 0.995
TARGET_UPDATE = 10
MEMORY_CAPACITY = int(1e5)
MAX_EPISODE_STEPS = 1000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")\

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


class DQN:
    def __init__(self, state_dim, action_dim):
        self.gamma = GAMMA ** N_STEP
        self.model = Agent.generate_model().to(DEVICE)
        self.target = Agent.generate_model().to(DEVICE)
        self.target.load_state_dict(self.model.state_dict())
        self.target.eval()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LR)
        self.memory = ReplayMemory(MEMORY_CAPACITY)

    def optimizer_step(self):
        if len(self.memory) < BATCH_SIZE:
            return
        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), device=DEVICE, dtype=torch.bool)
        non_final_next_states = torch.stack([s for s in batch.next_state if s is not None])
        state_batch = torch.stack(batch.state)
        action_batch = torch.stack(batch.action)
        reward_batch = torch.stack(batch.reward)

        state_action_values = self.model(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(BATCH_SIZE, device=DEVICE)
        next_state_values[non_final_mask] = self.target(non_final_next_states).max(1)[0].detach()
        expected_state_action_values = (next_state_values.unsqueeze(1) * GAMMA) + reward_batch
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.model.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

    def update(self, transition):
        state, action, next_state, reward, done = transition
        self.memory.push(torch.tensor(state, device=DEVICE),
                         torch.tensor([action], device=DEVICE),
                         torch.tensor(next_state, device=DEVICE) if not done else None,
                         torch.tensor([reward], device=DEVICE))
        self.optimizer_step()

    def act(self, state, target=False):
        with torch.no_grad():
            state = transform_state(state).to(DEVICE)
            return torch.argmax(self.model(state)).item()

    def save(self, i):
        torch.save(self.model.state_dict(), f'agent_{i}.pkl')

In [23]:
env = make("LunarLander-v2")
dqn = DQN(state_dim=8, action_dim=4)
episodes = 1000
scores = []
best_score = -1000.0
best_score_10 = -1000.0
total_steps = 0
start = time.time()
eps = EPS_START
for i in range(episodes):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    reward_buffer = deque(maxlen=N_STEP)
    state_buffer = deque(maxlen=N_STEP)
    action_buffer = deque(maxlen=N_STEP)
    while not done and steps < MAX_EPISODE_STEPS:
        #eps = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * total_steps / EPS_DECAY)
        total_steps += 1
        if random.random() < eps:
            action = env.action_space.sample()
        else:
            action = dqn.act(state)
        eps = max(EPS_END, EPS_DECAY * eps)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        steps += 1
        reward_buffer.append(reward)
        state_buffer.append(state)
        action_buffer.append(action)
        if len(reward_buffer) == N_STEP:
            dqn.update((state_buffer[0], action_buffer[0], next_state, sum([(GAMMA ** i) * r for i, r in enumerate(reward_buffer)]), done))
        state = next_state
        #env.render()
    scores.append(total_reward)
    if len(reward_buffer) == N_STEP:
        rb = list(reward_buffer)
        for k in range(1, N_STEP):
            dqn.update((state_buffer[k], action_buffer[k], next_state, sum([(GAMMA ** i) * r for i, r in enumerate(rb[k:])]), done))

    if (i + 1) % TARGET_UPDATE == 0:
        dqn.target.load_state_dict(dqn.model.state_dict())

    if (i + 1) % 50 == 0:
        current_score = np.mean(scores)
        print(f'Current score: {current_score}')
        scores = []
        if current_score > best_score:
            best_score = current_score
            dqn.save(50)
            print(f'Best model saved with score: {best_score}')
        end = time.time()
        elapsed = end - start
        start = end
        print(f'Elapsed time: {elapsed}')
    elif (i + 1) % 10 == 0:
        current_score_10 = np.mean(scores[-10:])
        print(f'Intermediate score: {current_score_10}')
        if current_score_10 > best_score_10:
          best_score_10 = current_score_10
          dqn.save(10)
          print(f'Best 10 model saved with score: {best_score_10}')

Intermediate score: -281.41897111817434
Best 10 model saved with score: -281.41897111817434
Intermediate score: -46.96905323592746
Best 10 model saved with score: -46.96905323592746
Intermediate score: -155.81274938006132
Intermediate score: -147.36453228998081
Current score: -141.74980542169703
Best model saved with score: -141.74980542169703
Elapsed time: 50.372435092926025
Intermediate score: 2.7922781639780228
Best 10 model saved with score: 2.7922781639780228
Intermediate score: -50.42879359993785
Intermediate score: -92.98412290708595
Intermediate score: -113.35472561136089
Current score: -69.64240826190857
Best model saved with score: -69.64240826190857
Elapsed time: 144.4490396976471
Intermediate score: -131.72478998131479
Intermediate score: -92.42112817424876
Intermediate score: -102.78507654976973
Intermediate score: -110.45663200333225
Current score: -108.33441977196375
Elapsed time: 144.95271754264832
Intermediate score: -71.43654401238464
Intermediate score: -55.653607718